# Create a biomiage.io model package

`bioimageio.core` implements functionality to create a zipped bioimage.io model. This zip can then be run in software that supports the bioimage.io specification and can be uploaded to [bioimage.io](https://bioimage.io/#/) in order to make it publicly available. For details on how to upload the model, see [these instructions](https://bioimage.io/docs/#/contribute_models/README).

Here, we will create three model packages:
- a toy model with torchscript weights
- a model based on another bioimage.io model where we add a post-processing step
- another version of this model where we add a new weight format to it compatible with deepImageJ

## Create a bioimage.io package for a new model

First, we create a zipped model package for a new model. Here, we use a very simple pytorch model and export it to torchscript. In practice you would use the weights for your trained model here. 

The procedure for creating a model with another weight format, e.g. keras, tensorflow or onnx, would be very similar.

In [ ]:
import os
import hashlib

# the imports for bioimage.io model export
import bioimageio.core
import numpy as np
import torch
import torch.nn as nn
from bioimageio.core.build_spec import build_model

In [ ]:
# create a temporary directory to store intermediate files
os.makedirs("my-model", exist_ok=True)

In [ ]:
# a very simple pytorch model: just a few convolutions
model = nn.Sequential(
    nn.Conv2d(1, 16, 3),
    nn.Conv2d(16, 32, 3),
    nn.Conv2d(32, 16, 3),
    nn.Conv2d(16, 1, 1)
)
model = torch.jit.script(model)

# save the model weights
model.save("my-model/weights.pt")

In [ ]:
# create test data for this model: an input image and a output image
# this data will be used for model test runs to ensure the model runs correctly and that the expected output can be reproduced
# NOTE: if you have pre-and-post-processing in your model (see the more advanced models for an example)
# you will need to save the input BEFORE preprocessing and the output AFTER postprocessing

input_ = np.random.rand(1, 1, 128, 128).astype("float32")  # an example input
np.save("my-model/test-input.npy", input_)
with torch.no_grad():
    output = model(torch.from_numpy(input_)).numpy()
np.save("my-model/test-output.npy", output)

In [ ]:
# create markdown documentation for your model
# this should describe how the model was trained, (and on which data)
# and also what to take into consideration when running the model, especially how to validate the model
# here, we just create a stub documentation
with open("my-model/doc.md", "w") as f:
    f.write("# My First Model\n")
    f.write("This model was trained on a very big dataset.\n")
    f.write("You should not let it get wet or feed it after midnight.\n")
    f.write("To validate its predictins, make sure that it does not produce any evil clones.\n")

In [ ]:
# now we can use the build_model function to create the zipped package.
# it takes the path to the weights and data we have just created, as well as additional information
# that will be used to add metadata to the rdf.yaml file in the model zip
# we only use a subset of the available options here, please refer to the advanced examples and to the
# function signature of build_model in order to get an overview of the full functionality
build_model(
    # the weight file and the type of the weights
    weight_uri="my-model/weights.pt",
    weight_type="torchscript",
    # the test input and output data as well as the description of the tensors
    # these are passed as list because we support multiple inputs / outputs per model
    test_inputs=["my-model/test-input.npy"],
    test_outputs=["my-model/test-output.npy"],
    input_axes=["bcyx"],
    output_axes=["bcyx"],
    # where to save the model zip, how to call the model and a short description of it
    output_path="my-model/model.zip",
    name="MyFirstModel",
    description="a fancy new model",
    # additional metadata about authors, licenses, citation etc.
    authors=[{"name": "Gizmo"}],
    license="CC-BY-4.0",
    documentation="my-model/doc.md",
    tags=["nucleus-segmentation"],  # the tags are used to make models more findable on the website
    cite=[{"text": "Gizmo et al.", "doi": "doi:10.1002/xyzacab123"}],
)

In [ ]:
# finally, we test that the expected outptus are reproduced when running the model.
# the 'test_model' function runs this test.
# it will output a list of dictionaries. each dict gives the status of a different test that is being run
# if all of them contain "status": "passed" then all tests were successful
from bioimageio.core.resource_tests import test_model
my_model = bioimageio.core.load_resource_description("my-model/model.zip") 
test_model(my_model)

## Advanced: Modify an existing bioimage.io model

Here, we modify an existing bioimage.io model by adding thresholding as a post-processing step to it.


In [ ]:
# we use a model from the webiste, please refer to the model_usage notebook for
# more details about this model and the general usage of the bioimageio.core library
doi = "10.5281/zenodo.6287342"
model_resource = bioimageio.core.load_resource_description(doi)

In [ ]:
# get the python file defining the architecture.
# this is only required for models with pytorch_state_dict weights
def get_architecture_source(rdf):
    # here, we need the raw resource, which contains the information from the resource description
    # before evaluation, e.g. the file and name of the python file with the model architecture
    raw_resource = bioimageio.core.load_raw_resource_description(rdf)
    # the python file defining the architecture for the pytorch weihgts
    model_source = raw_resource.weights["pytorch_state_dict"].architecture
    # download the source file if necessary
    source_file = bioimageio.core.resource_io.utils.resolve_source(
        model_source.source_file
    )
    # if the source file path does not exist, try combining it with the root path of the model
    if not os.path.exists(source_file):
        source_file = os.path.join(raw_resource.root_path, os.path.split(source_file)[1])
    assert os.path.exists(source_file), source_file
    class_name = model_source.callable_name
    return f"{source_file}:{class_name}"

In [ ]:
import xarray as xr
# we run prediction with the model once in order to get test outputs
input_image = np.load(model_resource.test_inputs[0])
input_array = xr.DataArray(input_image, dims=tuple(model_resource.inputs[0].axes))

with bioimageio.core.create_prediction_pipeline(model_resource) as prediction_pipeline:
    prediction = prediction_pipeline(input_array)[0]

In [ ]:
# create a subfolder to store the files for the new model
model_root = "./new_model"
os.makedirs(model_root, exist_ok=True)

# create the expected output tensor (= outputs thresholded at 0.5)
threshold = 0.5
new_output = prediction > threshold
new_output_path = f"{model_root}/new_test_output.npy"
np.save(new_output_path, new_output)

# add thresholding as post-processing procedure to our model
preprocessing = [[{"name": prep.name, "kwargs": prep.kwargs} for prep in inp.preprocessing] for inp in model_resource.inputs]
postprocessing = [[{"name": "binarize", "kwargs": {"threshold": threshold}}]]

# get the model architecture
# note that this is only necessary for pytorch state dict models
model_source = get_architecture_source(doi)

# we use the `parent` field to indicate that the new model is created based on
# the nucleus segmentation model we have obtained from bioimage.io
# this field is optional and only needs to be given for models that are created based on other models from bioimage.io
# the parent is specified via it's doi and the hash of its rdf file
model_root_folder = os.path.split(model_resource.weights["pytorch_state_dict"].source)[0]
rdf_file = os.path.join(model_root_folder, "rdf.yaml")
with open(rdf_file, "rb") as f:
    rdf_hash = hashlib.sha256(f.read()).hexdigest()
parent = {"uri": doi, "sha256": rdf_hash}

# the name of the new model and where to save the zipped model package
name = "new-model1"
zip_path = os.path.join(model_root, f"{name}.zip")

# `build_model` needs some additional information about the model, like citation information
# all this additional information is passed as plain python types and will be converted into the bioimageio representation internally  
# for more informantion, check out the function signature
# https://github.com/bioimage-io/core-bioimage-io-python/blob/main/bioimageio/core/build_spec/build_model.py#L252
cite = [{"text": cite_entry.text, "url": cite_entry.url} for cite_entry in model_resource.cite]

# the training data used for the model can also be specified by linking to a dataset available on bioimage.io
training_data = {"id": "ilastik/stradist_dsb_training_data"}

# the axes descriptions for the inputs / outputs
input_axes = ["bcyx"]
output_axes = ["bcyx"]

# the pytorch_state_dict weight file
weight_file = model_resource.weights["pytorch_state_dict"].source

# the path to save the new model with torchscript weights
zip_path = f"{model_root}/new_model2.zip"

# build the model! it will be saved to 'zip_path'
new_model_raw = build_model(
    weight_uri=weight_file,
    test_inputs=model_resource.test_inputs,
    test_outputs=[new_output_path],
    input_axes=input_axes,
    output_axes=output_axes,
    output_path=zip_path,
    name=name,
    description="nucleus segmentation model with thresholding",
    authors=[{"name": "Jane Doe"}],
    license="CC-BY-4.0",
    documentation=model_resource.documentation,
    covers=[str(cover) for cover in model_resource.covers],
    tags=["nucleus-segmentation"],
    cite=cite,
    parent=parent,
    architecture=model_source,
    model_kwargs=model_resource.weights["pytorch_state_dict"].kwargs,
    preprocessing=preprocessing,
    postprocessing=postprocessing,
    training_data=training_data,
)

In [ ]:
import napari
# helper function for showing multiple images in napari
def show_images(*images, names=None):
    v = napari.Viewer()
    for i, im  in enumerate(images):
        name = None if names is None else names[i]
        if isinstance(im, str):
            im = imageio.imread(im)
        v.add_image(im, name=name)

In [ ]:
# load the new model from the zipped package, run prediction and check the result
new_model = bioimageio.core.load_resource_description(zip_path)
with bioimageio.core.create_prediction_pipeline(new_model) as prediction_pipeline:
    prediction = prediction_pipeline(input_array)[0]
show_images(input_image, prediction, names=["input", "binarized-prediction"])

##  Create model compatible with deepImageJ

Now, we take the model from 10.5281/zenodo.6287342 again, convert its weights to torchscript and add the extra configuration needed to run it in deepImageJ.

In [ ]:
# `convert_weigths_to_pytorch_script` creates torchscript weigths based on the weights loaded from pytorch_state_dict
from bioimageio.core.weight_converter.torch import convert_weights_to_torchscript

# the path to save the newly created torchscript weights
weight_path = os.path.join(model_root, "weights.torchscript")
convert_weights_to_torchscript(new_model, weight_path)

# the path to save the new model with torchscript weights
zip_path = f"{model_root}/new_model2.zip"

In [ ]:
# build the new model with torchscript weights and with the deepimagej config
# we don't apply the post-processing here
new_model_raw = build_model(
    weight_uri=weight_path,
    weight_type="torchscript",
    test_inputs=model_resource.test_inputs,
    test_outputs=model_resource.test_outputs,
    input_axes=input_axes,
    output_axes=output_axes,
    output_path=zip_path,
    name=name,
    description="nucleus segmentation model with thresholding",
    authors=[{"name": "Jane Doe"}],
    license="CC-BY-4.0",
    documentation=model_resource.documentation,
    covers=[str(cover) for cover in model_resource.covers],
    tags=["nucleus-segmentation"],
    cite=cite,
    parent=parent,
    preprocessing=preprocessing,
    add_deepimagej_config=True
)

In [ ]:
# load the new model from the zipped package, run prediction and check the result
new_model = bioimageio.core.load_resource_description(zip_path)
with bioimageio.core.create_prediction_pipeline(new_model) as prediction_pipeline:
    prediction = prediction_pipeline(input_array)[0]
show_images(input_image, prediction, names=["input", "binarized-prediction"])